In [3]:
import pandas as pd
import numpy as np

In [4]:
#SET LINK TO RAW DATA HERE
raw_data_path = 'RAW DATA.xlsx'
prog_data = pd.read_excel(raw_data_path)
# get only the rows for summary table where the PROGRAM column is the institution TOTAL
summary_data = prog_data[prog_data['PROGRAM'] == 'INST TOTAL']
# get grand total
grand_total = prog_data[prog_data['INST'] == 'Grand Total']
# append grand total to end of summary data
summary_data = pd.concat([summary_data, grand_total], ignore_index=True)


In [5]:
# rename institution name of grand total rows to match inst name in raw data
def rename_grand_total_inst_name():
    #change the institution name to Grand total where institution is grand total in program data
    prog_data.loc[prog_data['INST'] ==
                  'Grand Total', 'INST_NAME'] = 'Grand Total'
    #change the institution name to Gross response rate where institution is grand total in summary data
    summary_data.loc[summary_data['INST'] ==
                     'Grand Total', 'INST_NAME'] = 'Gross Response Rate'

rename_grand_total_inst_name()


In [7]:
# sort the by program data by institution and if a row is a total of institution numbers
def rearrange_prog_data():
    # add new column is_total. True if row is the institution total
    prog_data['is_total'] = np.where(
        prog_data['PROGRAM'] == 'INST TOTAL', True, False)
    # add new column is_grand_total. True if row is the grand total
    prog_data['is_grand_total'] = np.where(
        prog_data['INST_NAME'] == 'Grand Total', True, False)
    # sort by is_grand_total (false rows first), inst name (alphabetical order), is_total (false rows first)
    prog_data.sort_values(
        by=['is_grand_total', 'INST_NAME', 'is_total'], inplace=True)
    #reset the index since the order of rows has changed
    prog_data.reset_index(drop=True, inplace=True)

rearrange_prog_data()


In [8]:
# get only the columns we're interested in for the by program table
prog_cols = [
    'INST_NAME', 'PROGRAM', 'Gross Frame', 'Survey Completes',
    'Gross Response Rate', 'Telephone Completes',
    'Telephone Response Rate', 'Web Completes', 'Web Response Rate',
    'To Reach Target', 'Cases Still in Calling Queue']
# exclude certain columns for summary table
summary_cols = [col for col in prog_cols if col != 'PROGRAM' and col !=
                'To Reach Target' and col != 'Cases Still in Calling Queue']
summary_data = summary_data[summary_cols]
prog_data = prog_data[prog_cols]


In [9]:
# rename the columns to match the ones in raw data
def rename_cols():
    summary_cols = {'INST_NAME': '2022 Baccalaureate Graduates Survey',
                    'Gross Frame': '2022 Cohort',
                    'Survey Completes': 'Total Number of Surveys',
                    'Telephone Completes': 'Number of Telephone Surveys',
                    'Web Completes': 'Number of Web Surveys'}
    prog_cols = {'INST_NAME': 'Institution / Program'}
    summary_data.rename(columns=summary_cols, inplace=True)
    prog_data.rename(columns=prog_cols, inplace=True)

rename_cols()


C:\Users\BHARRISO\AppData\Local\Temp\ipykernel_11148\1064090681.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prog_data.rename(columns=prog_cols, inplace=True)


In [10]:
writer = pd.ExcelWriter('Summary report tables.xlsx', engine='xlsxwriter')
# write out summary data in excel, but ignore the index column
summary_data.to_excel(writer, sheet_name='Summary', index=False)
# get workbook and worksheet
workbook = writer.book
worksheet = writer.sheets['Summary']


In [14]:
#autofit the width of the first column
def adjust_first_col_width(df):
    #get the first column
    first_col = summary_data.iloc[0:,0]
    #set the width as the row in the first column with the longest length
    first_col_width = max(first_col.astype(str).map(len).max(), len(first_col))
    #first column index in excel is zero by default
    col_idx = 0
    #set column width
    worksheet.set_column(col_idx, col_idx, first_col_width)

adjust_first_col_width(summary_data)


In [15]:
#change the header style to match the style in raw data
def change_header_style(df):
    cell_format = workbook.add_format()
    #set background and font color
    cell_format.set_bg_color('#366092')
    cell_format.set_font_color('white')
    #horizontally center text
    cell_format.set_align('center')
    #vertically center text
    cell_format.set_align('vcenter')
    #wrap text
    cell_format.set_text_wrap()
    #bolden text
    cell_format.set_bold()
    #write out each header with the format set above
    for i,col_name in enumerate(df.columns):
        worksheet.write(0, i, col_name, cell_format)

change_header_style(summary_data)

In [16]:
# format rate columns in summary table as percentage
def format_rates(df):
    #get only the columns where Rate is in the column name
    rate_cols = [col for col in df.columns if 'Rate' in col]
    # for each column name get it's index and store it in col_index
    col_index = [df.columns.get_loc(col) for col in rate_cols]
    # set the format to percentage
    format = workbook.add_format({'num_format': '0.0%'})
    # for each column change the format to percentage
    for ind in col_index:
        worksheet.set_column(ind, ind, None, format)

format_rates(summary_data)


In [17]:
# write out by program tables into excel worksheet
prog_data.to_excel(writer, sheet_name='By Program', index=False)
# open by program worksheet
worksheet = writer.sheets['By Program']
# format rate columns in by program tables
format_rates(prog_data)
#autofit first column width
adjust_first_col_width(prog_data)
#change header style
change_header_style(prog_data)


In [22]:
def collapse_prog_data():
    indexes = {}
    # by default the first excel row is 2
    starting_excel_row = 2
    # get all schools
    schools = summary_data['2022 Baccalaureate Graduates Survey']
    # filter out gross response rate from list of schools
    schools.drop(schools[schools == 'Gross Response Rate'].index, inplace=True)
    # filter out institution total rows
    no_totals = prog_data[prog_data['PROGRAM'] != 'INST TOTAL']
    # for each school store all the program rows as a dictionary of index lists
    for school in schools:
        indexes[school] = no_totals[no_totals['Institution / Program']
                                    == school].index.to_list()
    # iterate over each list in dictionary
    for idx_list in indexes.values():
        # TODO change double loop
        # iterate over each index in list
        for idx in idx_list:
            # set the row for each program index
            row = idx + starting_excel_row - 1
            # hide the row
            worksheet.set_row(row, None, None, {'level': 1, 'hidden': True})


collapse_prog_data()
writer.save()
